# SageMaker Training を利用して学習する

## 処理概要
1. local モードで学習ジョブを動かす
2. 推論が動くか確認する
2. Traning インスタンスでハイパーパラメータ最適化を用いた学習ジョブを動かす
3. 推論してみる

In [ ]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [ ]:
import os, sagemaker, yaml
from sagemaker.tensorflow import TensorFlow
import numpy as np
from matplotlib import pyplot as plt
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

role = sagemaker.get_execution_role()

with open('./setting.yaml', 'r') as yml:
    config = yaml.load(yml)
train_data_uri = config['train_data_uri']
test_data_uri = config['test_data_uri']
print(f'role: {role}')
print(f'train data uri: {train_data_uri}')
print(f'test data uri: {test_data_uri}')

## ローカルトレーニング
### 学習のための estimator 作成
* estimator とは training するためのインターフェース
* 学習スクリプトや、トレーニングに使用するコンピューティングリソース、バージョンなどを指定する

### 参考
[TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator)    
[TensorFlow Estimator の継承元のsagemaker.estimator.Framework](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework)    
[sagemaker.estimator.Framework の継承元のsagemaker.estimator.EstimatorBase](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)    

### ローカルトレーニングとは
* training 用のインスタンスではなく、Notebook インスタンスで学習を行う（ローカル＝jupyter 環境)
* training インスタンスの起動には時間がかかるため、学習コードにバグがあると、気づくのが10分後になる、などがありえる
    * 効率的にモデルの開発をするために、そのコードで動くかどうかを確認するためにローカルトレーニングがある
    * instance_type='local'と指定するだけで、ローカルトレーニングになる
    * トレーニングインスタンスで docker pull して学習を始められる
    * epochを1回など、動作確認の用途で利用する
* instance_type='ml.m5.xlarge'などインスタンスのタイプを指定すると training インスタンスでの学習になる

In [ ]:
local_estimator = TensorFlow(
    entry_point='./train_script/train.py', # 学習用コードの指定、git リポジトリを指定する方法もある。また、ディレクトリ以下のコードも利用したい場合はsource_dir を指定する
    role=role, # training ジョブを動かすロール。通常は notebook インスタンスに付与されている role を利用する
    instance_count=1,
    instance_type='local',
    framework_version='2.1.0',
    py_version='py3',
    hyperparameters={
        "epochs": 1,
        'dropout-rate':0.2
    }
)

In [ ]:
# 学習(3分強で完了)
%%time
local_estimator.fit({
    'train': train_data_uri,
    'test': test_data_uri
})

## 推論チェックのためのローカル predictor を作成
* 学習した結果、モデルをホスティングできるかのチェックを行う
* 学習したモデルをホスティングするためにpredictorがある
* estimator インスタンスから deploy するとpredictor を生成できる他、model のURIなどを指定して predictor を生成する方法もある(後述）
* training 同様、instance_type='local'を指定することでローカルのpredictorを生成できる

In [ ]:
# ローカルで学習したモデルで推論を行う
local_predictor = local_estimator.deploy(initial_instance_count=1, instance_type='local')

In [ ]:
# 動作テスト用に s3 にアップロードした npy ファイルを notebook インスタンスにダウンロードしてくる
!aws s3 cp {test_data_uri}/test_x.npy ./
!aws s3 cp {test_data_uri}/test_y.npy ./

In [ ]:
test_x = np.load('./test_x.npy')
text_y = np.load('./test_y.npy')
# データ確認
plt.imshow(test_x[0,:,:,0],'gray')

In [ ]:
# 推論確認（epochを1回しか回していないので外れていてもよい）
np.argmax(local_predictor.predict(test_x[0:1,:,:,:])['predictions'])

In [ ]:
# ローカルにホストしたい推論endpointを削除する
local_predictor.delete_endpoint()

## ハイパーパラメータ最適化を利用した training ジョブ
* 単純に学習するだけであれば、先程の instance_type = 'ml.m5.xlarge'など実在するトレーニングインスタンスを指定することで学習できる
* クラウドらしい使い方として、リソースを一時的に確保して HyperParameter Optimization を行う
    * HPO は先程同様Estimatorを作成し、探索する HyperParameter を設定する
    * 本ハンズオンは dropout 率を操作する
* スポットインスタンス を利用する
    * オンデマンドインスタンスはいつでも使えるインスタンスに対してスポットインスタンスは AWS 内で余っているリソースを安く利用するインスタンス
    * スポットインスタンスは余っているリソースなので、ジョブが途中で終了させられてしまう可能性がある
    * [詳細はこちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/model-managed-spot-training.html)
    * 利用にはuse_spot_instances = 'True'を指定するだけで利用可能

In [ ]:
hpo_estimator = TensorFlow(
    entry_point='./train_script/train.py',
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    framework_version='2.1.0',
    py_version='py3',
    hyperparameters={
        "epochs": 4
    },
    max_run = 5000, # 学習は最大で5000秒までにする設定
    use_spot_instances = 'True',
    max_wait = 300, # spotインスタンスの空きを待つ最大時間
)

## 探索するハイパーパラメータの設定
* [ContinuousParameter](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html#sagemaker.tuner.ContinuousParameter) は連続値を探索 ←今回はこちら
* [IntegerParameter](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html#sagemaker.tuner.IntegerParameter)は整数を探索
* [CategoricalParameter](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html#sagemaker.tuner.CategoricalParameter)はカテゴリーを探索

In [ ]:
hyperparameter_ranges = {'dropout-rate': ContinuousParameter(0.001, 0.8)} # dropout を 0.001 ～ 0.8 の間で探索

## ハイパーパラメータを探索する metrics を定義
* val_accuracy が最大化するような探索
* 学習字に標準出力される val_accuracy: 99.99 のような数値を取得する設定を正規表現で行う

In [ ]:
objective_metric_name = 'val_accuracy'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_accuracy',
                       'Regex': 'val_accuracy: ([0-9\\.]+)'}]

## ハイパーパラメータ探索ジョブを定義する
* 対象とする estimator,metrics,ジョブ数,並列数などを指定する    
[詳細はこちら](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html)

In [ ]:
tuner = HyperparameterTuner(hpo_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=4,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

## ハイパーパラメータ探索ジョブを実行する

In [ ]:
%%time
tuner.fit({
    'train': train_data_uri,
    'test': test_data_uri
})
tuner.wait()

## 学習した結果を hosting する
* 探索して一番よかった結果が tuner に残るので、deploy メソッドで hosting する
* instance_type と initial_instance_count を指定する    
[詳細はこちら](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html#sagemaker.tuner.HyperparameterTuner.deploy)

In [ ]:
predictor = tuner.deploy(instance_type='ml.m5.xlarge',initial_instance_count=1)

## 推論

In [ ]:
np.argmax(predictor.predict(test_x[0:1,:,:,:])['predictions'])

## hosting したものを削除
* delete_endpoint メソッドを呼び出すだけで削除可能

In [ ]:
tuner.delete_endpoint()

## 作成したモデルを再利用するために、モデルの保存先を設定ファイルに残す
* describe メソッドで詳細情報を出力できる

In [ ]:
best_training_job_name = tuner.describe()['BestTrainingJob']['TrainingJobName']

In [ ]:
sess = sagemaker.session.Session()
best_model_uri = sess.describe_training_job(best_training_job_name)['ModelArtifacts']['S3ModelArtifacts']
print(f'best_model_uri: {best_model_uri}')

In [ ]:
with open("./setting.yaml", mode='a') as f:
    f.write('best_model_uri: ' + best_model_uri +'\n')